In [1]:
import os
import os.path
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

2022-09-07 03:49:30.534009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 03:49:30.534027: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
x = []
y = []
file_cnt = 0
for dirpath, dirnames, filenames in os.walk("../Fall Data (New)/"):
    for filename in [f for f in filenames if f.endswith(".txt")]:
        df = pd.read_csv(os.path.join(dirpath, filename),header=None)
        fall_time = df[df[1].isna()]
        df = df[df[1].notna()]
        ax = np.array(df[1])
        ay = np.array(df[2])
        az = np.array(df[3])
        gx = np.array(df[4])
        gy = np.array(df[5])
        gz = np.array(df[6])
        time = np.array(df[0])
        a = np.sqrt(ax**2 + ay**2 + az**2)
        if filename[0] == "A":
            # ADL file
            file_cnt += 1
            for i in range(a.shape[0]):
                if a[i] >= 1.6:
                    st = i - 450
                    ed =  i + 450
                    if st < 0 or ed >= a.shape[0]:
                        continue

                    tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))


                    x.append(tmp)
                    y.append(1)
        elif filename[11] == "B":
            i = np.argmax(a)
            st = i - 450
            ed =  i + 450
            if st < 0 or ed >= a.shape[0] or a[i] < 1.6:
                print(filename,st,ed,a[i])
                continue
            file_cnt += 1
            tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))
            x.append(tmp)
            y.append(2)
            print("Same floor Fall:",filename)
            continue

        else:
            # Fall Data
            i = np.argmax(a)
            st = i - 450
            ed =  i + 450
            if st < 0 or ed >= a.shape[0] or a[i] < 1.6:
                print(filename,st,ed,a[i])
                continue
            file_cnt += 1
            print("Fall:",filename)
            tmp = np.array(list(zip(ax[st:ed],ay[st:ed],az[st:ed],gx[st:ed],gy[st:ed],gz[st:ed])))
            x.append(tmp)
            y.append(0)
print(file_cnt)

Fall: TESTING_ML_v58.txt
Fall: TESTING_ML_v55.txt
Fall: TESTING_ML_v49.txt
Fall: TESTING_ML_v9.txt
Fall: TESTING_ML_v19.txt
Fall: TESTING_ML_v29.txt
Fall: TESTING_ML_v57.txt
Fall: TESTING_ML_v25.txt
Fall: TESTING_ML_v50.txt
Fall: TESTING_ML_v23.txt
Fall: TESTING_ML_v30.txt
Fall: TESTING_ML_v56.txt
Fall: TESTING_ML_v53.txt
Fall: TESTING_ML_v28.txt
Fall: TESTING_ML_v40.txt
Fall: TESTING_ML_v16.txt
TESTING_ML_v20.txt -78 822 4.7132685049761385
Fall: TESTING_ML_v21.txt
Fall: TESTING_ML_v59.txt
Fall: TESTING_ML_v8.txt
Fall: TESTING_ML_v52.txt
Fall: TESTING_ML_v15.txt
Fall: TESTING_ML_v22.txt
Fall: TESTING_ML_v51.txt
Fall: TESTING_ML_v54.txt
TESTING_ML_v10.txt -102 798 12.53990430585497
Same floor Fall: TESTING_ML_BB_v89.txt
Same floor Fall: TESTING_ML_BB_v69.txt
Same floor Fall: TESTING_ML_BB_v97.txt
TESTING_ML_BB_v84.txt -384 516 1.5242047106606122
Same floor Fall: TESTING_ML_BB_v101.txt
Same floor Fall: TESTING_ML_BB_v96.txt
TESTING_ML_BB_v83.txt 16 916 12.494886954270536
Same floor Fall:

In [3]:
from collections import Counter
cnt =  Counter(y)
cnt

Counter({0: 24, 1: 9446, 2: 23})

In [4]:
X = np.array(x)
Y = np.array(y)
X = X.reshape(-1,900,6,1)
# X_flat = X.reshape(-1,5400)
print(X.shape,Y.shape)


(9493, 900, 6, 1) (9493,)


In [5]:
orig_shape = X.shape

wX = X.reshape(-1,5400)
print(X.shape,Y.shape,wX.shape)

(9493, 900, 6, 1) (9493,) (9493, 5400)


In [6]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
over = SMOTE(sampling_strategy='not majority')
under = RandomUnderSampler(sampling_strategy='majority')
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
new_X, new_Y = pipeline.fit_resample(wX, Y)

In [7]:
new_X = new_X.reshape((-1, 900, 6, 1))
print(new_X.shape,new_Y.shape)

(28338, 900, 6, 1) (28338,)


In [8]:
from collections import Counter
cnt =  Counter(new_Y)
cnt

Counter({0: 9446, 1: 9446, 2: 9446})

In [9]:
from sklearn.model_selection import train_test_split
new_X_train, new_X_test, new_y_train, new_y_test = train_test_split(new_X, new_Y, test_size=0.2, random_state=32)
new_y_train_cat = keras.utils.to_categorical(new_y_train)
new_y_test_cat = keras.utils.to_categorical(new_y_test)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
model = Sequential()
# 100%
# model.add(Conv2D(20, kernel_size=(50,6), activation='relu', input_shape=(900,6,1),strides=5)) 
model.add(Conv2D(7, kernel_size=(50,6), activation='relu', input_shape=(900,6,1),strides=5))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001,)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 171, 1, 7)         2107      
                                                                 
 flatten (Flatten)           (None, 1197)              0         
                                                                 
 dense (Dense)               (None, 3)                 3594      
                                                                 
Total params: 5,701
Trainable params: 5,701
Non-trainable params: 0
_________________________________________________________________


2022-09-07 03:49:38.390997: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 03:49:38.391325: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-07 03:49:38.391374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-09-07 03:49:38.391420: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-09-07 03:49:38.391465: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [11]:
model.fit(new_X_train, new_y_train_cat, validation_split=0.25, epochs=20,batch_size=128)
results = model.evaluate(new_X_test, new_y_test_cat, batch_size=128)

Epoch 1/20
133/133 [==============================] - 2s 14ms/step - loss: 0.0604 - accuracy: 0.9842 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 2/20
133/133 [==============================] - 2s 12ms/step - loss: 7.8924e-04 - accuracy: 1.0000 - val_loss: 5.2575e-04 - val_accuracy: 1.0000
Epoch 3/20
133/133 [==============================] - 2s 12ms/step - loss: 3.4883e-04 - accuracy: 1.0000 - val_loss: 2.7308e-04 - val_accuracy: 1.0000
Epoch 4/20
133/133 [==============================] - 2s 12ms/step - loss: 1.9998e-04 - accuracy: 1.0000 - val_loss: 1.6998e-04 - val_accuracy: 1.0000
Epoch 5/20
133/133 [==============================] - 2s 13ms/step - loss: 1.3078e-04 - accuracy: 1.0000 - val_loss: 1.1442e-04 - val_accuracy: 1.0000
Epoch 6/20
133/133 [==============================] - 2s 12ms/step - loss: 9.1966e-05 - accuracy: 1.0000 - val_loss: 8.4293e-05 - val_accuracy: 1.0000
Epoch 7/20
133/133 [==============================] - 2s 12ms/step - loss: 6.8286e-05 - accuracy: 1.00

In [12]:
from sklearn.metrics import confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=3372)
#print(X_test.shape)
predictions = model.predict(X, batch_size=128, verbose=0)
classes_x=(np.argmax(predictions,axis=1))
#print(predictions)
true_classes = Y

# y_test_flat = np.argmax(y_test, axis=-1)
c = confusion_matrix(Y, classes_x)
print('Confusion matrix:\n', c)
# print('Accuracy', )
print('Accuracy:{}%'.format((c[0, 0] + c[1, 1]) * 100 / (c[0, 1] + c[0, 0] + c[1, 1] + c[1, 0])))
print('Sensitivity:{}%'.format(c[0, 0]*100 / (c[0, 1] + c[0, 0])))
print('Specificity:{}%'.format(c[1, 1]*100 / (c[1, 1] + c[1, 0])))
# print('sensitivity', c[0, 0] / (c[0, 1] + c[0, 0]))
# print('specificity', c[1, 1] / (c[1, 1] + c[1, 0]))

Confusion matrix:
 [[  24    0    0]
 [   0 9446    0]
 [   0    0   23]]
Accuracy:100.0%
Sensitivity:100.0%
Specificity:100.0%


In [13]:
model.save('model/')


INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [14]:
converter = tf.lite.TFLiteConverter.from_saved_model('model/')
model_no_quant_tflite = converter.convert()

# Save the model to disk
open('model/model.tflite', "wb").write(model_no_quant_tflite)


2022-09-07 03:50:13.883116: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-09-07 03:50:13.883138: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-09-07 03:50:13.883595: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/
2022-09-07 03:50:13.884301: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-09-07 03:50:13.884315: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: model/
2022-09-07 03:50:13.886241: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-09-07 03:50:13.886829: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-09-07 03:50:13.911853: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: model/
2022-09-07 03:50:13.919829: I tensorf

24832

In [15]:
from tinymlgen import port
print(port(model, optimize=False))

INFO:tensorflow:Assets written to: /tmp/tmp597qsf8z/assets


INFO:tensorflow:Assets written to: /tmp/tmp597qsf8z/assets



#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

const unsigned char model_data[] DATA_ALIGN_ATTRIBUTE = {0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x88, 0x00, 0x00, 0x00, 0xe0, 0x00, 0x00, 0x00, 0x00, 0x5b, 0x00, 0x00, 0x10, 0x5b, 0x00, 0x00, 0x7c, 0x60, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x36, 0xa4, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x38, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76, 0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00,

2022-09-07 03:50:14.398035: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-09-07 03:50:14.398057: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-09-07 03:50:14.398165: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp597qsf8z
2022-09-07 03:50:14.398830: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-09-07 03:50:14.398843: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmp597qsf8z
2022-09-07 03:50:14.401550: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-09-07 03:50:14.426193: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmp597qsf8z
2022-09-07 03:50:14.434308: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 36145 m

In [34]:
import random
# while y[val] != 0:
# val = random.randrange(0,X.shape[0])
val = 8210
# print(val,X.shape)
test = (X[val].flatten())
%store test >"test.txt"

Writing 'test' (ndarray) to file 'test.txt'.
